Add stuff like installing datasets and stuf before running our app <br>
Also add different ways to run the process (Like run with completely new RNN weights, use pre-existing weights etc.)

In [20]:
import torch
from torch import nn
from ChildNetworkMLP import set_data_params,ChildModel,build_model_from_encoding,train_model

from Controller import RNNController,generate_sequence,VOCAB_SIZE
from torch.utils.data import DataLoader


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
from utils import check_cifar_dataset_exists
data_path=check_cifar_dataset_exists()

In [9]:
data=torch.load(data_path+'cifar/train_data.pt')
label=torch.load(data_path+'cifar/train_label.pt')

C:\Users\vaibh\AppData\Local\Temp\ipykernel_10304\4193000220.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data=torch.load(data_path+'cifar/train_data.pt')
C:\Users\va

In [10]:
input_dim = data[0].view(1,-1).size()[1]
output_dim = torch.max(label).item()+1

print(input_dim)
print(output_dim)

3072
10


In [11]:
controllerModel = RNNController(VOCAB_SIZE)

In [12]:
model_encoding = generate_sequence(controllerModel)

model_encoding

['START', '4', '6', '6', '5', '9', 'END']

In [13]:
# TODO: Sanity checks before passing to generate child network

In [14]:
set_data_params(input_dim,output_dim)

In [15]:
childModel = ChildModel(model_encoding)

3072


In [16]:
childModel.state_dict()

OrderedDict([('model.0.weight',
              tensor([[-3.2630e-05,  7.9310e-03, -9.2707e-03,  ...,  9.4738e-03,
                       -8.5941e-03, -8.5626e-03],
                      [-1.7861e-02, -1.7127e-02, -9.2436e-03,  ..., -8.0382e-03,
                       -1.4878e-02,  1.1514e-02],
                      [-1.2221e-03,  6.7321e-04, -1.0859e-02,  ...,  6.0114e-04,
                       -1.7120e-02,  3.5651e-03],
                      [-1.6501e-02,  5.4949e-03,  1.5598e-02,  ..., -1.3396e-02,
                       -1.2899e-02, -1.5286e-02]])),
             ('model.0.bias', tensor([ 0.0116, -0.0151,  0.0024,  0.0158])),
             ('model.2.weight',
              tensor([[-0.0077,  0.4005,  0.3747,  0.2474],
                      [ 0.4000,  0.2943,  0.0352, -0.0277],
                      [ 0.3987,  0.2512,  0.3546,  0.1762],
                      [-0.1102,  0.4543,  0.2596,  0.3583],
                      [ 0.1932, -0.0447,  0.3401, -0.0728],
                      [-0.2086, 

In [19]:
criterion = nn.CrossEntropyLoss() # Cross-entropy loss function
optimizer = torch.optim.Adam(childModel.parameters(), lr=0.001) # 0.001 is the learning rate

In [24]:
train_loader = DataLoader(data, batch_size=128, shuffle=True) 



tensor([[[[0.2314, 0.1686, 0.1961,  ..., 0.6196, 0.5961, 0.5804],
          [0.0627, 0.0000, 0.0706,  ..., 0.4824, 0.4667, 0.4784],
          [0.0980, 0.0627, 0.1922,  ..., 0.4627, 0.4706, 0.4275],
          ...,
          [0.8157, 0.7882, 0.7765,  ..., 0.6275, 0.2196, 0.2078],
          [0.7059, 0.6784, 0.7294,  ..., 0.7216, 0.3804, 0.3255],
          [0.6941, 0.6588, 0.7020,  ..., 0.8471, 0.5922, 0.4824]],

         [[0.2431, 0.1804, 0.1882,  ..., 0.5176, 0.4902, 0.4863],
          [0.0784, 0.0000, 0.0314,  ..., 0.3451, 0.3255, 0.3412],
          [0.0941, 0.0275, 0.1059,  ..., 0.3294, 0.3294, 0.2863],
          ...,
          [0.6667, 0.6000, 0.6314,  ..., 0.5216, 0.1216, 0.1333],
          [0.5451, 0.4824, 0.5647,  ..., 0.5804, 0.2431, 0.2078],
          [0.5647, 0.5059, 0.5569,  ..., 0.7216, 0.4627, 0.3608]],

         [[0.2471, 0.1765, 0.1686,  ..., 0.4235, 0.4000, 0.4039],
          [0.0784, 0.0000, 0.0000,  ..., 0.2157, 0.1961, 0.2235],
          [0.0824, 0.0000, 0.0314,  ..., 0

In [22]:
train_loss, train_time = train_model(childModel, train_loader, criterion, optimizer, device)

print(train_loss, train_time)

ValueError: too many values to unpack (expected 2)